# Day5_0: 결측치 처리 (Missing Data)

## 📚 학습 목표

**Part 1: 기초**
1. 결측치 패턴 이해하기
2. isna()와 notna()로 결측치 확인하기
3. fillna()로 결측치 채우기
4. dropna()로 결측치 제거하기
5. interpolate()로 결측치 보간하기

**Part 2: 심화**
1. 결측치 패턴 분석하기
2. px.imshow()로 결측치 시각화하기
3. 다양한 채우기 전략 (평균, 중앙값, ffill, bfill) 적용하기
4. 결측치 처리 전후 비교 시각화하기

---

## 🎯 왜 이것을 배우나요?

| 개념 | 실무 활용 | 예시 |
|------|----------|------|
| 결측치 확인 | 데이터 품질 진단 | 고객 정보 누락 파악 |
| 결측치 채우기 | 분석 가능한 데이터 확보 | 설문조사 미응답 처리 |
| 결측치 제거 | 불완전 데이터 제외 | 주요 필드 누락 행 삭제 |
| 결측치 시각화 | 패턴 파악 | 특정 컬럼에만 결측치 집중 발견 |

**분석가 관점**: 실무 데이터의 70%는 결측치를 포함하며, 적절한 처리 없이는 분석이 불가능합니다!

---

In [ ]:
# 필요한 라이브러리 임포트
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

# 경고 메시지 숨기기
import warnings
warnings.filterwarnings('ignore')

print("라이브러리 임포트 완료!")

# Part 1: 기초

---

## 1.1 결측치란?

### 결측치(Missing Data)의 개념

결측치는 **데이터가 없는 상태**를 의미합니다. pandas에서는 `NaN` (Not a Number), `None`, `pd.NA` 등으로 표현됩니다.

In [ ]:
# 결측치가 포함된 고객 데이터 생성
customer_data = {
    '고객명': ['김철수', '이영희', '박민수', '최지영', '정다은'],
    '나이': [28, np.nan, 35, 42, np.nan],
    '구매금액': [150000, 200000, np.nan, 180000, 95000],
    '이메일': ['kim@email.com', None, 'park@email.com', 'choi@email.com', None],
    '등급': ['골드', '실버', '골드', np.nan, '브론즈']
}

df = pd.DataFrame(customer_data)
print("고객 데이터:")
print(df)

### 결측치가 발생하는 이유

1. **데이터 수집 누락**: 설문조사 미응답, 센서 고장
2. **정보 미제공**: 개인정보 입력 거부, 선택 항목
3. **데이터 병합 오류**: JOIN 시 매칭 실패
4. **시스템 오류**: 로그 기록 실패, DB 에러

---

## 1.2 결측치 확인하기

### isna()와 notna()

- `isna()`: 결측치면 True, 아니면 False
- `notna()`: 결측치가 아니면 True, 결측치면 False

In [ ]:
# 결측치 확인 (True/False 테이블)
print("결측치 위치 (True = 결측치):")
print(df.isna())

In [ ]:
# 컬럼별 결측치 개수
print("컬럼별 결측치 개수:")
print(df.isna().sum())

In [ ]:
# 전체 결측치 개수
total_missing = df.isna().sum().sum()
print(f"전체 결측치 개수: {total_missing}개")

In [ ]:
# 결측치 비율 계산
missing_ratio = (df.isna().sum() / len(df)) * 100
print("컬럼별 결측치 비율(%):")
print(missing_ratio.round(2))

### 💡 실무 예시: 데이터 품질 리포트

데이터 수집 후 첫 단계로 결측치를 파악하여 데이터 품질을 평가합니다.

In [ ]:
# 데이터 품질 요약
quality_report = pd.DataFrame({
    '컬럼명': df.columns,
    '전체_행수': len(df),
    '결측치_수': df.isna().sum().values,
    '결측치_비율(%)': (df.isna().sum() / len(df) * 100).values.round(2)
})

print("📊 데이터 품질 리포트:")
print(quality_report)

---

## 1.3 fillna()로 결측치 채우기

### 기본 사용법

`fillna(값)`을 사용하여 결측치를 특정 값으로 채울 수 있습니다.

In [ ]:
# 모든 결측치를 0으로 채우기
df_filled_zero = df.fillna(0)
print("결측치를 0으로 채운 결과:")
print(df_filled_zero)

In [ ]:
# 특정 값으로 채우기
df_filled_text = df.fillna('미입력')
print("결측치를 '미입력'으로 채운 결과:")
print(df_filled_text)

### 컬럼별로 다른 값 채우기

In [ ]:
# 컬럼별 채우기 값 지정
fill_values = {
    '나이': 30,  # 평균 나이로 추정
    '구매금액': 0,  # 구매 이력 없음
    '이메일': '미제공',
    '등급': '일반'
}

df_filled_custom = df.fillna(fill_values)
print("컬럼별로 다르게 채운 결과:")
print(df_filled_custom)

### 💡 실무 예시: 설문조사 미응답 처리

설문조사에서 선택 항목 미응답은 '응답 안 함'으로, 필수 항목은 평균값으로 채웁니다.

In [ ]:
# 설문조사 데이터
survey = pd.DataFrame({
    '응답자': ['A', 'B', 'C', 'D', 'E'],
    '만족도': [5, 4, np.nan, 5, np.nan],
    '추천의향': ['예', '아니오', '예', np.nan, '예']
})

print("원본 설문조사:")
print(survey)
print()

# 처리: 만족도는 평균, 추천의향은 '응답 안 함'
avg_satisfaction = survey['만족도'].mean()
survey_filled = survey.fillna({
    '만족도': avg_satisfaction,
    '추천의향': '응답 안 함'
})

print("처리 후:")
print(survey_filled)

---

## 1.4 dropna()로 결측치 제거하기

### 행 단위 제거

`dropna()`는 결측치가 포함된 행을 제거합니다.

In [ ]:
# 결측치가 하나라도 있는 행 제거
df_dropped = df.dropna()
print("결측치 포함 행 제거 후:")
print(df_dropped)
print(f"\n남은 행 수: {len(df_dropped)}행 (원본: {len(df)}행)")

### 특정 컬럼 기준으로 제거

In [ ]:
# 구매금액이 결측인 행만 제거
df_dropped_amount = df.dropna(subset=['구매금액'])
print("구매금액 결측 행 제거:")
print(df_dropped_amount)

### 임계값 설정

In [ ]:
# 최소 3개 이상의 값이 있는 행만 유지
df_thresh = df.dropna(thresh=3)
print("최소 3개 값이 있는 행만 유지:")
print(df_thresh)

### 💡 실무 예시: 핵심 필드 누락 데이터 제외

고객 분석 시 나이, 구매금액 등 핵심 필드가 없는 데이터는 분석에서 제외합니다.

In [ ]:
# 핵심 필드: 나이, 구매금액
core_fields = ['나이', '구매금액']
df_valid = df.dropna(subset=core_fields)

print("핵심 필드 완전한 데이터만 유지:")
print(df_valid)
print(f"\n분석 가능한 고객 수: {len(df_valid)}명")

---

## 1.5 interpolate()로 결측치 보간하기

### 선형 보간 (Linear Interpolation)

`interpolate()`는 앞뒤 값을 기준으로 **선형적으로 결측치를 추정**합니다. 시계열 데이터나 연속된 숫자 데이터에 유용합니다.

In [ ]:
# 시계열 판매량 데이터 (일부 결측)
sales_data = pd.DataFrame({
    '날짜': pd.date_range('2024-01-01', periods=10, freq='D'),
    '판매량': [100, 120, np.nan, 160, np.nan, 200, 210, np.nan, 240, 250]
})

print("원본 판매량 데이터:")
print(sales_data)

In [ ]:
# 선형 보간 적용
sales_data['판매량_보간'] = sales_data['판매량'].interpolate()
print("\n선형 보간 후:")
print(sales_data)

### 다양한 보간 방법

In [ ]:
# 여러 보간 방법 비교
values = [10, np.nan, np.nan, 40, 50]
comparison = pd.DataFrame({
    '원본': values,
    'linear': pd.Series(values).interpolate(method='linear'),
    'quadratic': pd.Series(values).interpolate(method='quadratic'),
    'cubic': pd.Series(values).interpolate(method='cubic')
})

print("보간 방법 비교:")
print(comparison)

### 💡 실무 예시: 센서 데이터 보간

온도 센서가 일시적으로 고장났을 때, 앞뒤 측정값을 기준으로 값을 추정합니다.

In [ ]:
# 온도 센서 데이터 (일부 누락)
sensor_data = pd.DataFrame({
    '시간': pd.date_range('2024-01-01 00:00', periods=12, freq='H'),
    '온도': [20.5, 21.0, np.nan, np.nan, 23.0, 23.5, np.nan, 24.5, 25.0, 25.2, 25.5, 26.0]
})

# 선형 보간으로 복원
sensor_data['온도_복원'] = sensor_data['온도'].interpolate()

print("센서 데이터 복원:")
print(sensor_data[['시간', '온도', '온도_복원']])

---

# Part 2: 심화

---

## 2.1 결측치 패턴 분석

### 결측치 패턴 유형

1. **MCAR** (Missing Completely At Random): 완전 무작위 결측
2. **MAR** (Missing At Random): 다른 변수와 관련된 결측
3. **MNAR** (Missing Not At Random): 결측값 자체와 관련된 결측

In [ ]:
# 복잡한 실무 데이터 시뮬레이션
np.random.seed(42)
n = 200

complex_data = pd.DataFrame({
    '고객ID': range(1, n+1),
    '나이': np.random.randint(20, 70, n),
    '소득': np.random.randint(2000, 10000, n) * 10000,
    '구매금액': np.random.randint(50, 500, n) * 1000,
    '만족도': np.random.randint(1, 6, n)
})

# 의도적으로 패턴 있는 결측치 생성
# 1) 고소득자는 소득 정보를 안 알려주는 경향 (MNAR)
complex_data.loc[complex_data['소득'] > 8000000, '소득'] = np.nan

# 2) 나이 많은 사람은 만족도 응답 안 하는 경향 (MAR)
complex_data.loc[complex_data['나이'] > 60, '만족도'] = np.nan

# 3) 구매금액 일부는 완전 랜덤 결측 (MCAR)
random_missing = np.random.choice(complex_data.index, size=20, replace=False)
complex_data.loc[random_missing, '구매금액'] = np.nan

print("결측치 패턴이 있는 데이터:")
print(complex_data.head(10))

In [ ]:
# 컬럼별 결측치 요약
missing_summary = pd.DataFrame({
    '결측치_수': complex_data.isna().sum(),
    '결측치_비율(%)': (complex_data.isna().sum() / len(complex_data) * 100).round(2)
})

print("결측치 요약:")
print(missing_summary)

### 💡 실무 예시: 결측치 패턴에 따른 처리 전략

- **MCAR**: 제거 또는 평균값 대체 가능
- **MAR**: 다른 변수 기반 예측값 사용
- **MNAR**: 별도 카테고리로 처리 (예: '미공개')

---

## 2.2 px.imshow()로 결측치 시각화

### 결측치 히트맵

결측치를 **히트맵**으로 시각화하면 패턴을 한눈에 파악할 수 있습니다.

In [ ]:
# 결측치를 1, 값 있는 것을 0으로 변환
missing_matrix = complex_data.isna().astype(int)

# Plotly imshow로 시각화
fig = px.imshow(
    missing_matrix.T,  # 전치하여 컬럼이 y축에 오도록
    labels=dict(x="행 번호", y="컬럼명", color="결측 여부"),
    title="📊 결측치 패턴 히트맵 (흰색 = 결측치)",
    color_continuous_scale=['#2ecc71', '#e74c3c'],  # 초록(값 있음), 빨강(결측)
    aspect='auto'
)

fig.update_layout(height=400)
fig.show()

### 결측치 비율 막대 그래프

In [ ]:
# 컬럼별 결측치 비율 계산
missing_pct = (complex_data.isna().sum() / len(complex_data) * 100).sort_values(ascending=False)

fig = px.bar(
    x=missing_pct.index,
    y=missing_pct.values,
    labels={'x': '컬럼명', 'y': '결측치 비율 (%)'},
    title="📊 컬럼별 결측치 비율",
    color=missing_pct.values,
    color_continuous_scale='Reds'
)

fig.update_layout(showlegend=False, height=400)
fig.show()

### 💡 실무 예시: 데이터 품질 대시보드

결측치 시각화를 통해 데이터 수집 프로세스의 문제점을 빠르게 파악합니다.

---

## 2.3 다양한 채우기 전략

### 평균, 중앙값, 최빈값

숫자형 데이터의 결측치는 **통계값**으로 채울 수 있습니다.

In [ ]:
# 테스트 데이터
test_df = pd.DataFrame({
    '점수': [85, 90, np.nan, 78, np.nan, 95, 88, np.nan, 92]
})

# 평균, 중앙값으로 채우기
test_df['평균값'] = test_df['점수'].fillna(test_df['점수'].mean())
test_df['중앙값'] = test_df['점수'].fillna(test_df['점수'].median())

print("통계값으로 채우기:")
print(test_df)

### Forward Fill (ffill)과 Backward Fill (bfill)

- `ffill()`: 이전 값으로 채우기
- `bfill()`: 다음 값으로 채우기

In [ ]:
# 시계열 재고 데이터
inventory = pd.DataFrame({
    '날짜': pd.date_range('2024-01-01', periods=8),
    '재고': [100, np.nan, np.nan, 85, np.nan, 80, 75, np.nan]
})

# Forward fill과 Backward fill 비교
inventory['재고_ffill'] = inventory['재고'].ffill()
inventory['재고_bfill'] = inventory['재고'].bfill()

print("Forward/Backward Fill 비교:")
print(inventory)

### 그룹별 채우기

카테고리별로 다른 값으로 채울 수 있습니다.

In [ ]:
# 지점별 매출 데이터
branch_sales = pd.DataFrame({
    '지점': ['서울', '서울', '부산', '부산', '대구', '대구'],
    '매출': [100, np.nan, 80, np.nan, 90, np.nan]
})

# 지점별 평균으로 채우기
branch_sales['매출_채움'] = branch_sales.groupby('지점')['매출'].transform(
    lambda x: x.fillna(x.mean())
)

print("지점별 평균으로 채우기:")
print(branch_sales)

### 💡 실무 예시: 전략적 결측치 처리

- **나이**: 평균 또는 중앙값
- **구매금액**: 0 (구매 안 함)
- **등급**: 최빈값 또는 '일반'
- **시계열 데이터**: interpolate() 또는 ffill()

In [ ]:
# 원본 고객 데이터에 전략적 처리 적용
df_strategic = df.copy()

# 나이: 중앙값
df_strategic['나이'] = df_strategic['나이'].fillna(df_strategic['나이'].median())

# 구매금액: 0
df_strategic['구매금액'] = df_strategic['구매금액'].fillna(0)

# 이메일: '미제공'
df_strategic['이메일'] = df_strategic['이메일'].fillna('미제공')

# 등급: '일반'
df_strategic['등급'] = df_strategic['등급'].fillna('일반')

print("전략적 처리 후:")
print(df_strategic)

---

## 2.4 결측치 처리 전후 비교 시각화

### Before/After 비교

In [ ]:
# 처리 전후 결측치 개수 비교
before = df.isna().sum()
after = df_strategic.isna().sum()

comparison = pd.DataFrame({
    '컬럼': before.index,
    '처리_전': before.values,
    '처리_후': after.values
})

# 막대 그래프로 비교
fig = go.Figure()

fig.add_trace(go.Bar(
    x=comparison['컬럼'],
    y=comparison['처리_전'],
    name='처리 전',
    marker_color='#e74c3c'
))

fig.add_trace(go.Bar(
    x=comparison['컬럼'],
    y=comparison['처리_후'],
    name='처리 후',
    marker_color='#2ecc71'
))

fig.update_layout(
    title="📊 결측치 처리 전후 비교",
    xaxis_title="컬럼명",
    yaxis_title="결측치 개수",
    barmode='group',
    height=400
)

fig.show()

### 분포 비교 (처리 전후)

결측치 처리 후 데이터 분포가 어떻게 변했는지 확인합니다.

In [ ]:
# 구매금액 분포 비교 (결측치 제외 vs 0으로 채움)
fig = go.Figure()

# 처리 전 (결측치 제외)
fig.add_trace(go.Histogram(
    x=df['구매금액'].dropna(),
    name='처리 전 (결측치 제외)',
    opacity=0.7,
    marker_color='#e74c3c'
))

# 처리 후 (0으로 채움)
fig.add_trace(go.Histogram(
    x=df_strategic['구매금액'],
    name='처리 후 (0으로 채움)',
    opacity=0.7,
    marker_color='#2ecc71'
))

fig.update_layout(
    title="📊 구매금액 분포 비교",
    xaxis_title="구매금액 (원)",
    yaxis_title="빈도",
    barmode='overlay',
    height=400
)

fig.show()

### 💡 실무 예시: 처리 방법에 따른 통계 변화

결측치 처리 방법에 따라 평균, 표준편차 등이 달라질 수 있으므로 신중하게 선택해야 합니다.

In [ ]:
# 처리 방법별 통계 비교
original_mean = df['구매금액'].mean()
dropna_mean = df['구매금액'].dropna().mean()
fillna_zero_mean = df['구매금액'].fillna(0).mean()
fillna_mean_mean = df['구매금액'].fillna(df['구매금액'].mean()).mean()

stats_comparison = pd.DataFrame({
    '처리_방법': ['원본(결측 포함)', '결측치 제거', '0으로 채움', '평균으로 채움'],
    '평균_구매금액': [original_mean, dropna_mean, fillna_zero_mean, fillna_mean_mean]
})

print("처리 방법별 평균 구매금액:")
print(stats_comparison)

---

## 🎯 실습 퀴즈

**난이도**: ⭐ (쉬움) ~ ⭐⭐⭐⭐⭐ (어려움)

---

### Q1. 결측치 개수 확인하기 ⭐

**문제**: 다음 데이터에서 각 컬럼의 결측치 개수를 출력하세요.

```python
data = pd.DataFrame({
    'A': [1, 2, np.nan, 4],
    'B': [np.nan, 2, 3, np.nan],
    'C': [1, 2, 3, 4]
})
```

**기대 결과**: A: 1개, B: 2개, C: 0개

In [ ]:
# 여기에 코드를 작성하세요


### Q2. 특정 값으로 채우기 ⭐

**문제**: 다음 데이터의 모든 결측치를 -1로 채우세요.

```python
scores = pd.Series([85, np.nan, 90, np.nan, 78])
```

In [ ]:
# 여기에 코드를 작성하세요


### Q3. 결측치 포함 행 제거 ⭐⭐

**문제**: 다음 데이터에서 결측치가 하나라도 있는 행을 제거하세요.

```python
df = pd.DataFrame({
    'name': ['A', 'B', 'C', 'D'],
    'age': [25, np.nan, 30, 35],
    'score': [90, 85, np.nan, 95]
})
```

**기대 결과**: 2개 행 남음 (A, D)

In [ ]:
# 여기에 코드를 작성하세요


### Q4. 컬럼별로 다르게 채우기 ⭐⭐

**문제**: 다음 데이터에서 'age'는 평균값으로, 'city'는 '미입력'으로 결측치를 채우세요.

```python
df = pd.DataFrame({
    'name': ['김철수', '이영희', '박민수'],
    'age': [25, np.nan, 30],
    'city': ['서울', '부산', np.nan]
})
```

In [ ]:
# 여기에 코드를 작성하세요


### Q5. 선형 보간 적용 ⭐⭐

**문제**: 다음 시계열 데이터의 결측치를 선형 보간으로 채우세요.

```python
temps = pd.Series([20, np.nan, np.nan, 26, 28])
```

**기대 결과**: 20, 22, 24, 26, 28

In [ ]:
# 여기에 코드를 작성하세요


---

## Part 2 심화 퀴즈

---

### Q6. 결측치 비율 계산 ⭐⭐⭐

**문제**: 다음 데이터에서 각 컬럼의 결측치 비율(%)을 계산하고, 20% 이상인 컬럼만 출력하세요.

```python
df = pd.DataFrame({
    'A': [1, np.nan, 3, np.nan, 5, np.nan, 7, 8, 9, 10],
    'B': [np.nan, 2, 3, 4, 5, 6, 7, 8, 9, 10],
    'C': [1, 2, 3, np.nan, 5, 6, np.nan, 8, np.nan, 10]
})
```

In [ ]:
# 여기에 코드를 작성하세요


### Q7. Forward Fill 적용 ⭐⭐⭐

**문제**: 다음 재고 데이터의 결측치를 forward fill로 채우세요.

```python
inventory = pd.Series([100, np.nan, np.nan, 85, np.nan, 80], 
                      index=pd.date_range('2024-01-01', periods=6))
```

**기대 결과**: 100, 100, 100, 85, 85, 80

In [ ]:
# 여기에 코드를 작성하세요


### Q8. 그룹별 평균으로 채우기 ⭐⭐⭐⭐

**문제**: 다음 지점별 매출 데이터에서 각 지점의 평균 매출로 결측치를 채우세요.

```python
sales = pd.DataFrame({
    '지점': ['서울', '서울', '서울', '부산', '부산', '부산'],
    '매출': [100, np.nan, 120, 80, np.nan, 90]
})
```

**힌트**: `groupby().transform()`을 사용하세요.

In [ ]:
# 여기에 코드를 작성하세요


### Q9. 복합 조건 결측치 처리 ⭐⭐⭐⭐

**문제**: 다음 요구사항에 맞춰 결측치를 처리하세요:
1. 'age' 컬럼: 중앙값으로 채우기
2. 'salary' 컬럼: 직급(position)별 평균으로 채우기
3. 'city' 컬럼: '미입력'으로 채우기

```python
employees = pd.DataFrame({
    'name': ['A', 'B', 'C', 'D', 'E', 'F'],
    'age': [25, np.nan, 30, np.nan, 28, 32],
    'position': ['사원', '사원', '대리', '대리', '과장', '과장'],
    'salary': [3000, np.nan, 4000, np.nan, 5000, 5200],
    'city': ['서울', '부산', np.nan, '대구', np.nan, '인천']
})
```

In [ ]:
# 여기에 코드를 작성하세요


### Q10. 결측치 패턴 시각화 및 처리 전략 ⭐⭐⭐⭐⭐

**문제**: 다음 작업을 수행하세요:
1. 결측치 히트맵을 px.imshow()로 그리기
2. 각 컬럼의 결측치 비율을 계산하여 막대 그래프로 표현
3. 적절한 전략으로 결측치 처리 (컬럼 특성에 맞게)
4. 처리 전후 결측치 개수를 비교하는 그래프 생성

```python
np.random.seed(42)
survey_data = pd.DataFrame({
    '응답자ID': range(1, 21),
    '나이': np.random.choice([25, 30, 35, np.nan], 20),
    '만족도': np.random.choice([1, 2, 3, 4, 5, np.nan], 20),
    '재구매의향': np.random.choice(['예', '아니오', np.nan], 20),
    '추천점수': np.random.choice([7, 8, 9, 10, np.nan], 20)
})
```

In [ ]:
# 여기에 코드를 작성하세요


---

## 📊 학습 정리

### Part 1: 기초 핵심 요약

| 메소드 | 기능 | 실무 활용 |
|--------|------|----------|
| `isna()` | 결측치 확인 (True/False) | 데이터 품질 진단 |
| `fillna(값)` | 결측치를 특정 값으로 채우기 | 설문조사 미응답 처리 |
| `dropna()` | 결측치 포함 행 제거 | 핵심 필드 누락 데이터 제외 |
| `interpolate()` | 선형 보간으로 채우기 | 센서 데이터, 시계열 복원 |

### Part 2: 심화 핵심 요약

| 기법 | 방법 | 언제 쓰나? |
|------|------|----------|
| 결측치 시각화 | `px.imshow(df.isna())` | 패턴 파악, 데이터 품질 리포트 |
| 평균/중앙값 채우기 | `fillna(df.mean())` | 숫자형 데이터, MCAR 패턴 |
| Forward/Backward Fill | `ffill()`, `bfill()` | 시계열 데이터 |
| 그룹별 채우기 | `groupby().transform()` | 카테고리별 다른 전략 적용 |

### 💡 실무 팁

1. **결측치 파악 먼저**: 분석 전 반드시 `df.isna().sum()`으로 현황 파악
2. **컬럼 특성 고려**: 숫자는 평균/중앙값, 카테고리는 최빈값 또는 '미입력'
3. **제거 vs 채우기**: 결측치가 5% 미만이면 제거, 20% 이상이면 채우기 고려
4. **시계열은 보간**: 시간 순서가 있는 데이터는 `interpolate()` 또는 `ffill()` 사용
5. **처리 후 검증**: 분포 변화, 통계값 변화를 시각화로 확인
6. **문서화**: 어떤 전략으로 처리했는지 기록 (재현 가능성)
7. **별도 컬럼 생성**: 원본은 유지하고 '_채움' 등 새 컬럼에 저장